# 搜索
- 搜索可以最原始的like的方式进行搜索，但是对于大量的数据该搜索会很慢。所以需要使用搜索引擎来加快搜索速度。搜索引擎会将所有需要的数据使用算法做一个索引，以后搜索的时候只需要根据这个索引即可找到相应的数据，搜索引擎做索引的过程会比较慢，但是一旦索引建立完成，那么以后在搜索的时候就会很快了

# django-haystack插件
- 这个插件是由django提供搜索功能的，它提供了一个搜索的接口，底层可以根据自己的需求更换搜索引擎，它有点类似django中的orm插件，提供了一个操作数据库的接口，但是底层具体使用哪个数据库是可以自己设置的，安装通过：pip install django-haystack

## 注意（一定注意）：
- 搜索引擎的关键词是通过form标签提交得到的，无论是通过HTTP请求还是ajax请求，给request传递的变量名应该是q。即input标签的属性name='q',因为这牵涉到面搜索引擎是通过q来获取input中输入的值的

## 搜索引擎
- django-haystack它支持的的搜索引擎有Solr，Elasticsearch，whoosh，Xapian等，whoosh是基于纯python的搜索引擎，搜索速度快，集成方便，pip install whoosh

## 集成步骤
- 1.在项目中安装django-haystack
    - 在install_apps中加入：'haystack'
- 2.设置搜索引擎
    - 在settings中加入以下配置：同时在项目目录下新建一个文件包：whoosh_index
            HAYSTACK_CONNECTIONS = {
                'default': {
                    # 设置haystack的搜索引擎
                    'ENGINE': 'haystack.backends.whoosh_backend.WhooshEngine',
                    # 设置索引文件的位置
                    'PATH': os.path.join(BASE_DIR, 'whoosh_index'),
                }
            }
            
- 3.创建索引类
    - 在模型所属的APP下创建一个search_indexes.py文件(名字只能为这个名字，且必须放在要创建索引的APP的下面)，然后创建索引类，比如要给模型News创建索引，代码如下：
        from haystack import indexes
        from .models import News

        class NewsIndex(indexes.SearchIndex, indexes.Indexable):
            text = indexes.CharField(focument=True, use_template=True)

            def get_model(self):
                return News

            def index_queryset(self, using=None):
                return self.get_model().objects.all()
                
- 4.添加模板
    - 在templates文件夹下新建一个文件夹，名为search(必须是这个名字),在它下面新建文件夹indexes，又在indexex下新建news（即app名字），在news（即模型名对应的文件夹）下新建一个名为模型名字加上在search_indexes中定义的字段名组合的txt文件，形如：news_text.txt,然后将要索引的字段写入txt文件中，格式为{{ object.title }} {{ object.content }},其中title和content是你要给模型建立索引的字段名字
    - 目录结构如下：
            templates
                search
                    indexes
                        news(app的名字)
                            news(模型的名字，小写）_text.txt

    - 接着在search文件夹下创建search.html模板文件(必须要是该文件名，haystack会自动寻找它)，haystack会自动的在templates文件夹下寻找这个模板文件渲染，并且会给这个模板文件传入page,paginator,query等参数，其中page和Paginator分别是django内置的Page类和Paginator类的对象，query查询的关键字，可以通过page.object_list获取到查出来的数据

- 5.添加映射
    - 已经不能用原来那种映射格式了
        - 原来的：path('search/', views.search, name='search')
        - 现在的：path('search/', include('haystack.urls')),
        
- 6.建立索引
    - 在项目的根目录下，使用命令python manage.py rebuild_index创建索引
    - 如果不想每次数据增删查改后都要手动的创建索引，可以在settings.py中配置：
            #增删查改后自动创建索引
            HAYSTACK_SIGNAL_PROCESSOR = 'haystack.signals.RealtimeSignalProcessor'
- 6.其他
    - 在HTML中不能直接使用news.来查找属性了，而应该是使用page.object_list
                                        {% for result in page.object_list %}
                                {% with result.object as news %}
                                    <li>
                                        <a class="li-container" href="{% url 'news:detail' news_id=news.id %}"
                                           target="_blank">
                                            <div class="li-img">
                                                <img src="{{ news.thumbnail }}" alt="图片加载失败">
                                            </div>
                                            <div class="li-content">
                                                <div class="li-title">{{ news.title }}</div>
                                                <div class="li-detail">{{ news.descs }}</div>
                                                <div class="li-other">
                                                    <span class="li-type"><label>{{ news.category.name }}</label></span>
                                                    <span class="li-time">
                                                    {{ news.pub_time|time_format }}
                                                </span>
                                                    <span class="li-author">
                                                    {{ news.author.username }}
                                                <i class="li-author-name"></i>
                                                </span>
                                                </div>
                                            </div>
                                        </a>
                                    </li>
                                {% endwith %}
                            {% endfor %}

    - 更改链接：
        - 原来的a href="{% url 'search' %}">搜索</a>
        - 现在的：a href="/search/">搜索</a>
    - 注：尽管上面的配置好了，但是这个对中文的搜索并不友好，搜不出来
    
## 使用jieba分词替换Whoosh默认的分词
- whoosh默认是采用正则表达式进行分词的，这对于英文来说是足够的，但是对于中文不友好，因此要替换为jieba分词。jieba分词是中文分词中最好用的免费的分词库，安装：pip install jieba
- 替换步骤：
    - 1.完成安装后，拷贝django-env/lib/site-packages/haystack/backends/whoosh_backend.py其中的代码，然后在要设置索引的APP下面创建一个名叫whoosh_cn_backend.py文件，把刚刚复制的代码粘贴进去，然后看第二步
    - 2.分两种情况
        - 如果使用jieba自带的解析器，那么就在whoosh_cn_backend.py文件中导入语句的最后导入from jieba.analyse import ChineseAnalyzer,然后将其中的SteemingAnalyzer全部替换为ChineseAnalyzer。
        - 如果想自定义ChineseAnalyzer，那么在whoosh_cn_backend.py中的靠前写入以下代码，然后还是将其中的SteemingAnalyzer全部替换为ChineseAnalyzer。
                import jieba
                from whoosh.analysis import Token, Tokenizer

                class ChineseTokenizers(Tokenizer):
                    def __call__(self, value, positions=False, chars=False,
                                 keeporiginal=False, removestops=True,
                                 start_pos=0, start_char=0, mode="", **kwargs):
                        t = Token(positions, chars, removestops=removestops, mode=mode, **kwargs)
                        seglist = jieba.cut(value, cut_all=True)
                        for w in seglist:
                            t.original = t.text = w
                            t.boost = 1.0
                            if positions:
                                t.pos = start_pos + value.find(w)
                            if chars:
                                t.startchar = start_char + value.find(w)
                                t.endchar = start_char + value.find(w) + len(w)
                            yield t


                def ChineseAnalyzer():
                    return ChineseTokenizers()
    - 3.配置新的路径：
            # 使用jieba分词以后应当使用如下设置
            HAYSTACK_CONNECTIONS = {
                'default': {
                    # 设置haystack的自定义搜索引擎，即whoosh_cn_backend的路径，即它所在的app的路径
                    'ENGINE': 'apps.news.whoosh_cn_backend.WhooshEngine',
                    # 设置索引文件的位置
                    'PATH': os.path.join(BASE_DIR, 'whoosh_index'),
                }
            }
            # 为了避免每次数据更新都要设置新的路径，在settings中如下配置：
            # 添加此项，当数据库改变时，会自动更新索引，非常方便
            HAYSTACK_SIGNAL_PROCESSOR = 'haystack.signals.RealtimeSignalProcessor'
    - 4.创建索引
        - 在终端输入：rebuild_index即可